https://hostadvice.com/blog/ai/types-of-ollama-models/

In [ ]:
from langchain_ollama import OllamaLLM
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report

Pré-processamento

In [15]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

print(f"Test set: {len(test)} exemplos, classes: {test['classe'].value_counts().to_dict()}")
#train_oversampling = pd.read_csv('train_oversampling.csv')

Test set: 1858 exemplos, classes: {0: 1273, -1: 417, 1: 168}


LLM

In [16]:
'''escolher entre:
Meta Llama 3.1 8B-Instruct
Qwen 3-8
JetMoE-8B
FuxiTranyu-8B
GPT-J-6B'''

'escolher entre:\nMeta Llama 3.1 8B-Instruct\nQwen 3-8\nJetMoE-8B\nFuxiTranyu-8B\nGPT-J-6B'

In [ ]:
import pandas as pd
from langchain_ollama import OllamaLLM
from sklearn.metrics import classification_report, accuracy_score, f1_score
import re

llm = OllamaLLM(model="llama3.1:8b", temperature=0.1)

def extract_label(response):

    print(response)
    resp = str(response).strip().lower()
    
    #se o modelo classificar como deve
    for num in ['-1', '0', '1']:
        if num in resp:
            return int(num)
    
    #fallback caso o modelo se passe 
    if any(word in resp for word in ['citação', 'citacao']):
        return -1
    if any(word in resp for word in ['facto', 'fato']):
        return 0
    if any(word in resp for word in ['viés', 'vies', 'tendencioso']):
        return 1
    
    return 0


def generate_predictions(test_df, prompt_template, max_examples=5):
    predictions = []

    examples = train.sample(max_examples).to_dict('records') if max_examples > 0 else []

    for idx, row in test_df.iterrows():
        text = row['sentences']
        true_label = row['classe']

        prompt = prompt_template.format(
            text=text,
            examples=examples,
            label_map="-1: Citação | 0: Facto | 1: Viés"
        )

        response = llm.invoke(prompt)
        pred_label = extract_label(response)

        predictions.append({
            'id': idx,
            'text': text,
            'true_label': true_label,
            'predicted_label': pred_label,
            'raw_response': str(response),
            'prompt_used': prompt_template[:100] + "..."
        })

    return pd.DataFrame(predictions)


Zero-Shot

In [27]:

zero_shot_prompt = """
Tu és um classificador de texto. Classifica o seguinte texto em português brasileiro:
{label_map}

Texto: {text}

Responde APENAS com -1, 0 ou 1 (sem mais texto):
"""

zero_shot_results = generate_predictions(test, zero_shot_prompt, max_examples=0)

print(zero_shot_results[['true_label','predicted_label']].head())
print()
print(classification_report(zero_shot_results['true_label'],
                            zero_shot_results['predicted_label'],
                            digits=3))
print("Accuracy:", accuracy_score(zero_shot_results['true_label'],
                                 zero_shot_results['predicted_label']))
print("F1 (weighted):", f1_score(zero_shot_results['true_label'],
                                 zero_shot_results['predicted_label'],
                                 average='weighted'))


0
0
-1
0
0
0
0
-1
0
-1
0: Facto
-1
-1
0
-1
0: Facto
0
0
0
0
0
0
0
-1
0
0
-1
0
0: Facto
-1
0: Facto
0
0
-1
0
0: Facto
-1
-1
0
0
0
0
0
0
-1
0
0
-1
0
0
0
0
0
-1
0
-1
0
-1
0
0
-1
0
-1
0
0
0
0
-1
0
0
0
0
0: Facto
0
-1
-1
-1
-1
0
0
0
0
0
0
0
0
-1
0
0
0
0
0: Facto
0
-1
0
0
0
0
0: Facto
0
0: Facto
0
-1
-1
0
0
0: Facto
Classificação: 0 (Facto)
0
-1
0
0
-1
0
0
0: Facto
0
-1
0
-1
0
0
0
0: Facto
0
0
0
-1. A resposta parece ser uma reação pessoal e emocional à entrada do usuário, em vez de um fato objetivo ou uma opinião imparcial.
-1: Citação
0: Facto
0
-1
-1
-1
-1
-1
0
0
0
0
0
0
-1
0: Facto
0: Facto
Classificação: 1 (Viés)

Justificativa: O texto parece ser uma solicitação para que o leitor responda a uma pesquisa do Ipec, mas sem fornecer contexto ou informações adicionais. A falta de informação e a abordagem direta podem indicar um viés na apresentação da questão.
0
-1
0
0
0
0
0
-1
0
0
-1
0
0
0
0
-1
0
0
-1
0
-1
0
0
0
0
0
0: Facto
0
0
0
-1
0
0
0
0: Facto
-1
0: Facto
-1
0
0
0
0
0
0
0
0
0
0
0
0
0


Few-shot

Chain of thought

Zero shot chain of thought

Self consistency(?)

Model 2